Caixeiro viajante
=================



## Introdução



Até agora, sempre que nós aplicamos os operadores de `cruzamento` ou de `mutação` nós não nos preocupamos se o indivíduo gerado por estes processo era um `indivíduo válido`. Um indivíduo válido é aquele que representa uma solução possível e bem formulada para o problema em questão.

Por exemplo, no problema das caixas binárias, [1, 0, 0, 1] é um indivíduo válido para o caso de termos 4 caixas. Um exemplo de `indivíduo inválido` para este mesmo problema seria [1, 0, 0, a], pois um dos genes está assumindo um valor fora do domínio. Outro exemplo de indivíduo inválido poderia ser [1, 1, 0], pois é um indivíduo com apenas 3 genes, sendo que o esperado eram 4 genes.

Neste experimento nós veremos estratégias para evitar que indivíduos inválidos sejam obtidos quando usamos os operadores de cruzamento e de mutação. No notebook seguinte veremos como aplicar uma penalidade para indivíduos inválidos que forem gerados durante uma busca genética com restrições.



## Objetivo



Encontrar uma solução para o problema do caixeiro viajante. Considere que ele irá visitar 5 cidades, pode iniciar sua viagem por qualquer uma destas cidades e deve retornar à cidade de início. Durante seu trajeto, não pode visitar a mesma cidade duas vezes (única exceção é a cidade inicial).



## Descrição do problema



O problema consiste em descobrir a rota de menor distância entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos.

O código abaixo é um gerador de coordenadas de cidades. Use ele caso queira.



In [1]:
def cria_cidades(n):
    """Cria um dicionário aleatório de cidades com suas posições (x,y).

    Argumentos
      n: inteiro positivo
        Número de cidades que serão visitadas pelo caixeiro.

    Retorno:
      Dicionário contendo o nome das cidades como chaves e a coordenada no plano
      cartesiano das cidades como valores.
    """

    cidades = {}

    for i in range(n):
        cidades[f"Cidade {i}"] = (rd.random(), rd.random())

    return cidades

## Importações



In [2]:
import random
from itertools import permutations

from funcoes import cria_cidades

from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv
from funcoes import funcao_objetivo_cv
from funcoes import selecao_torneio_min as funcao_selecao # esse já temos!
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

## Códigos e discussão



Nesse experimento temos a oportunidade de abordar o problema do caixeiro viajante e entender um pouco sobre como evitar que indivíduos inválidos sejam gerados no nosso problema.

Sabendo que o caixeiro quer fazer o menos caminho possível, esse problema também trata-se de um problema de minimização em que há uma restrição, já que o caixeiro não pode visitar a mesma cidade duas vezes, com a exceção de que ele volta à primeira cidade após visitar todas as outras.

Dessa forma, as funções importadas foram basicamente as mesmas do experimento passado, mas podemos ver algumas novidades, como a função `cria_cidades`, criada acima, que nos retorna uma lista de cidade e suas respectivas coordenadas, que serão muito úteis para o nosso problema.

Outra novidade nas importações trata-se da importação da função `permutations`, ela será utilizada nos operadores de cruzamento e mutação, pois permitirá que façamos as operações sem retornar indivíduos ou cidades repetidas, cumprindo os requisitos do problema!

In [3]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 5
CIDADES = cria_cidades(NUMERO_DE_CIDADES)

In [4]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_objetivo_individuo(individuo):
    return funcao_objetivo_cv(individuo, CIDADES)

Tendo isso em mente, a estratégia que seguiremos para a `mutação` no caso desse experimento trata-se da mutação de troca, que consiste na troca do valor de dois genes entre si.

Dessa forma, conseguimos mudar os nossos genes sem a necessidade de remover ou acrescentar quaisquer cidades que sejam, mas apenas alterar a ordem das visitas.

Já a estratégia a ser utilizada para o `cruzamento` será o cruzamento ordenado, um método em que os filhos mantém os mesmos genes que os pais tinham, mas em uma ordem diferente.

Esse método é muito útil em problemas como esse em que a ordem dos genes é importante e em que não devemos alterar os genes em si.


In [5]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES)

melhor_fitness_ja_visto = float("inf") #infinito em python

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador+2
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:
        posicao  = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness

Como o problema do caixeiro viajante pode ser considerado um problema NP difícil, só conseguimos, de fato, testar se uma determinada rota é a melhor possível se todas as rotas forem testadas.

Dessa forma, achamos interessante propor também uma solução por busca exaustiva (dado que nesse caso ainda é possível fazê-la sem exigir tanto dos nossos computadores) e comparar o resultado obtido dessa forma com o obtido por meio de algoritmos genéticos. 

In [6]:
# Busca testando todas as permutações

melhor_fitness_ever = float("inf")

# testando todas as permutações possíveis 
for caminho in permutations(list(CIDADES.keys())):
    distancia = funcao_objetivo_individuo(caminho)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

In [7]:
# Chacando os resultados

print()
print("Melhor individuo obtido por algoritmos genéticos")
print(melhor_individuo_ja_visto, "com distancia:", melhor_fitness_ja_visto)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)


Melhor individuo obtido por algoritmos genéticos
['Cidade 0', 'Cidade 2', 'Cidade 1', 'Cidade 3', 'Cidade 4'] com distancia: 2.616113591279454

Melhor individuo obtido por busca exaustiva:
('Cidade 0', 'Cidade 2', 'Cidade 1', 'Cidade 3', 'Cidade 4') com distância: 2.616113591279454


## Conclusão



Portanto, podemos perceber que sendo o nosso objetivo encontrar uma solução para o problema do caixeiro viajante em que não houvesse repetição de cidade e que a distância fosse a menor possível; ele foi atingido com sucesso!

Isso pode ser observado, pois no resultado obtido não houve repetição e a resposta encontrada através dos algoritmos genéticos(AG) foi exatamente igual à obtida por busca exaustiva(BE). Acredito que isso tenha sido possível porque o número de possibilidades ainda era pequeno, mas quanto maior o número de possibilidades, maior a probabilidade da resposta obtida por AG ser menos exata e mais custoso computacionalmente será obter a resposta exata por BE.

Nesse experimento aprendemos, então, sobre um problema de minimização com restrições, no qual foi preciso alterar um pouco as funções cruzamento e mutação para contornar a problemática. A construção desse raciocínio foi muito importante e com certeza será útil em problemas e experimentos futuros!!&#x1F604;

## Playground

